<a href="https://colab.research.google.com/github/chaithanya21/Tensorflow-In-Practice/blob/master/Transfer_Learning_on_Cats_vs_Dogs_Kaggle_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0.0

In [0]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.0.0


In [3]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

--2020-01-21 08:14:27--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c12::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   155MB/s    in 0.4s    

2020-01-21 08:14:27 (155 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

In [0]:
local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

In [6]:

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [7]:
base_model=tf.keras.applications.inception_v3.InceptionV3(input_shape=(150,150,3),
                                                          include_top=False,weights='imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [8]:
print(len(base_model.layers))

311


In [9]:
#Summarry
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [0]:
base_model.trainable=False

In [11]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [0]:
prediction_layer=tf.keras.layers.Dense(1,activation='sigmoid')

In [0]:
model=tf.keras.models.Sequential([
                                  base_model,
                                  global_average_layer,
                                  prediction_layer
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 21,804,833
Trainable params: 2,049
Non-trainable params: 21,802,784
_________________________________________________________________


In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [17]:
model.fit_generator(train_generator,steps_per_epoch=100,validation_data=validation_generator,epochs=20,verbose=2,validation_steps=50)

Epoch 1/20
100/100 - 546s - loss: 0.5874 - accuracy: 0.6710 - val_loss: 0.2606 - val_accuracy: 0.9100
Epoch 2/20
100/100 - 541s - loss: 0.5222 - accuracy: 0.7330 - val_loss: 0.2297 - val_accuracy: 0.9320
Epoch 3/20
100/100 - 505s - loss: 0.4890 - accuracy: 0.7535 - val_loss: 0.2345 - val_accuracy: 0.9340
Epoch 4/20
100/100 - 500s - loss: 0.4679 - accuracy: 0.7735 - val_loss: 0.2182 - val_accuracy: 0.9380
Epoch 5/20
100/100 - 498s - loss: 0.4544 - accuracy: 0.7780 - val_loss: 0.1901 - val_accuracy: 0.9440
Epoch 6/20
100/100 - 497s - loss: 0.4427 - accuracy: 0.7945 - val_loss: 0.2153 - val_accuracy: 0.9450
Epoch 7/20
100/100 - 521s - loss: 0.4247 - accuracy: 0.8000 - val_loss: 0.2444 - val_accuracy: 0.9450
Epoch 8/20
100/100 - 551s - loss: 0.4063 - accuracy: 0.8095 - val_loss: 0.4213 - val_accuracy: 0.9200
Epoch 9/20
100/100 - 525s - loss: 0.4393 - accuracy: 0.7955 - val_loss: 0.2210 - val_accuracy: 0.9450
Epoch 10/20
100/100 - 529s - loss: 0.4254 - accuracy: 0.7980 - val_loss: 0.2749 - 